- https://www.kaggle.com/datasets/amitabhajoy/bengaluru-house-price-data 

## 1. Data Loading and Preprocessing.

In these steps, we will load the Bengaluru House Data dataset using pandas and perform an initial exploration to understand its structure and contents.

- **Data Import:** The dataset is loaded into a pandas DataFrame named `data`.
- **Shape:** The dataset contains 12,530 rows and 7 columns after initial cleaning.
- **Columns:**  
    - `location`: Area or locality of the property  
    - `size`: Number of bedrooms (e.g., "2 BHK", "4 Bedroom")  
    - `total_sqft`: Total area in square feet  
    - `bath`: Number of bathrooms  
    - `price`: Price of the property (in lakhs)  
    - `bhk`: Extracted number of bedrooms as integer  
    - `price_per_sqft`: Price per square foot

We will also check for missing values, data types, and unique values in key columns to guide further cleaning and preprocessing steps. This foundational understanding helps in identifying potential issues such as outliers, inconsistent data, and the need for encoding categorical variables.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("Bengaluru_House_Data.csv")

In [3]:
data.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


We can see there are 9 columns with different values area_type , availability, location size socitey , toatl_sqft, balcony and price.


In [4]:
data.shape

(13320, 9)

As above we can see that the dataset have 13320 rows and 9 columns.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          13304 non-null  object 
 4   society       7818 non-null   object 
 5   total_sqft    13320 non-null  object 
 6   bath          13247 non-null  float64
 7   balcony       12711 non-null  float64
 8   price         13320 non-null  float64
dtypes: float64(3), object(6)
memory usage: 936.7+ KB


### Data Overview and Missing Values

The dataset initially contains 9 columns, with 6 columns of object type and 3 columns of float type. Below is a summary of missing values in each column:

- **location**: 1 missing value  
- **size**: 16 missing values  
- **society**: Many missing values (over 5,000)  
- **bath**: 73 missing values  
- **balcony**: Approximately 600 missing values  

To handle these missing values, we can either replace them with appropriate statistics (mean, median, or mode) or drop columns that are not important for our analysis.

#### Column-wise Data Types and Non-Null Counts

| Column         | Non-Null Count | Data Type |
|----------------|---------------|-----------|
| area_type      | 13,320        | object    |
| availability   | 13,320        | object    |
| location       | 13,319        | object    |
| size           | 13,304        | object    |
| society        | 7,818         | object    |
| total_sqft     | 13,320        | object    |
| bath           | 13,247        | float64   |
| balcony        | 12,711        | float64   |
| price          | 13,320        | float64   |

We will proceed by cleaning the data, handling missing values, and dropping columns that are not relevant for further analysis.

In [6]:
for col in data.columns:
    print(data[col].value_counts())
    print("*"*20)
# The above loop prints the value counts for each column in the DataFrame,
# helping to understand the distribution and frequency of unique values in every column.

area_type
Super built-up  Area    8790
Built-up  Area          2418
Plot  Area              2025
Carpet  Area              87
Name: count, dtype: int64
********************
availability
Ready To Move    10581
18-Dec             307
18-May             295
18-Apr             271
18-Aug             200
                 ...  
16-Oct               1
17-Jan               1
16-Nov               1
16-Jan               1
14-Jul               1
Name: count, Length: 81, dtype: int64
********************
location
Whitefield                         540
Sarjapur  Road                     399
Electronic City                    302
Kanakpura Road                     273
Thanisandra                        234
                                  ... 
3rd Stage Raja Rajeshwari Nagar      1
Chuchangatta Colony                  1
Electronic City Phase 1,             1
Chikbasavanapura                     1
Abshot Layout                        1
Name: count, Length: 1305, dtype: int64
********************
siz

In [7]:
data.isna().sum()

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

### Null Value Analysis & Next Steps

Several columns (`location`, `size`, `bath`, `balcony`, `society`) had missing values. To ensure data quality:

- Filled missing values in key columns using median (for numerics) or mode (for categoricals).
- Dropped columns with excessive nulls or low relevance.
- Converted data types and engineered new features (`bhk`, `price_per_sqft`).
- Removed outliers and standardized location categories.

These steps prepare the dataset for effective analysis and modeling.


In [8]:
data.shape

(13320, 9)

In [9]:
data.drop(columns=['area_type', 'availability', 'society', 'balcony'], inplace=True)

In [10]:
data.shape

(13320, 5)

### Dropping Irrelevant or Redundant Columns

We remove the columns `area_type`, `availability`, `society`, and `balcony` from the dataset because:

- They have many missing values or low relevance to price prediction.
- They introduce unnecessary complexity or high cardinality (e.g., `society`).
- Their information is either redundant or not useful for our analysis and modeling.

This step simplifies the dataset and focuses on the most relevant features.

In [11]:
data.describe()

,bath,price
count,13247.000000,13320.000000
mean,2.692610,112.565627
std,1.341458,148.971674
min,1.000000,8.000000
25%,2.000000,50.000000
50%,2.000000,72.000000
75%,3.000000,120.000000
max,40.000000,3600.000000


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    13319 non-null  object 
 1   size        13304 non-null  object 
 2   total_sqft  13320 non-null  object 
 3   bath        13247 non-null  float64
 4   price       13320 non-null  float64
dtypes: float64(2), object(3)
memory usage: 520.4+ KB


In [13]:
data['location'].value_counts()

location
Whitefield                         540
Sarjapur  Road                     399
Electronic City                    302
Kanakpura Road                     273
Thanisandra                        234
                                  ... 
3rd Stage Raja Rajeshwari Nagar      1
Chuchangatta Colony                  1
Electronic City Phase 1,             1
Chikbasavanapura                     1
Abshot Layout                        1
Name: count, Length: 1305, dtype: int64


There was only one missing value in the `location` column, which was filled with `'Whitefield'`, the most frequent location. This maintains data consistency.

In [14]:
data['location'] = data['location'].fillna('Whitefield')


In [15]:
data['size'].value_counts()

size
2 BHK         5199
3 BHK         4310
4 Bedroom      826
4 BHK          591
3 Bedroom      547
1 BHK          538
2 Bedroom      329
5 Bedroom      297
6 Bedroom      191
1 Bedroom      105
8 Bedroom       84
7 Bedroom       83
5 BHK           59
9 Bedroom       46
6 BHK           30
7 BHK           17
1 RK            13
10 Bedroom      12
9 BHK            8
8 BHK            5
11 BHK           2
10 BHK           2
11 Bedroom       2
27 BHK           1
19 BHK           1
43 Bedroom       1
16 BHK           1
14 BHK           1
12 Bedroom       1
13 BHK           1
18 Bedroom       1
Name: count, dtype: int64

In [16]:
data['size'] = data['size'].fillna('2 BHK')

The `size` column has only 16 missing values, which are filled with the most frequent value, `2 BHK`, to maintain consistency.

In [17]:
data['bath'].median() # bathroom has 73 null values so we will replace them my median value.

np.float64(2.0)

In [18]:
data['bath'] = data['bath'].fillna(data['bath'].median())

Filled missing values in the `bath` column with the median number of bathrooms to maintain consistency and minimize the impact of outliers.

In [19]:
data['bath'].isna().sum()

np.int64(0)

In [20]:
data['bhk'] = data['size'].str.split().str.get(0).astype(int)

Extracting Number of Bedrooms (`bhk`)

The `bhk` column is generated by extracting the numeric value from the `size` column (e.g., "2 BHK" becomes 2). This transformation standardizes the number of bedrooms as an integer feature, making it easier to analyze and model.

Converting categorical or object-type columns into numeric values (such as integers or floats) is a crucial preprocessing step. It enables more effective statistical analysis and machine learning, as most algorithms require numerical input.

In [21]:
data['bhk'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 13320 entries, 0 to 13319
Series name: bhk
Non-Null Count  Dtype
--------------  -----
13320 non-null  int64
dtypes: int64(1)
memory usage: 104.2 KB


we can see there is no null value now.

In [22]:
data['bhk'].value_counts()

bhk
2     5544
3     4857
4     1417
1      656
5      356
6      221
7      100
8       89
9       54
10      14
11       4
27       1
19       1
16       1
43       1
14       1
12       1
13       1
18       1
Name: count, dtype: int64

Checking for outliers.


In [23]:
data[data.bhk>20]

,location,size,total_sqft,bath,price,bhk
1718,2Electronic City Phase II,27 BHK,8000,27.0,230.0,27
4684,Munnekollal,43 Bedroom,2400,40.0,660.0,43


only two columns that  have value greter than 20 so they are outliers.

Now let's check for `total_sqft` column.

In [24]:
data['total_sqft'].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      shape=(2117,), dtype=object)

To handle `total_sqft` values given as ranges (e.g., "1200-1500"), we take their mean. For other values, we convert them directly to float. This ensures all entries in `total_sqft` are numeric and consistent for analysis.

In [25]:
def convertRange(x):
    temp = x.split('-')
    if len(temp) == 2:
        return (float(temp[0]) + float(temp[1]))/2
    try:
        return float(x)
    except:
        return None

In [26]:
data['total_sqft'] = data['total_sqft'].apply(convertRange)

In [27]:
data['total_sqft'].dtype

dtype('float64')

In [28]:
data.head()

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3
4,Kothanur,2 BHK,1200.0,2.0,51.00,2


Price Per square feet

In [29]:
data[['price', 'total_sqft']].corr()

,price,total_sqft
price,1.000000,0.575559
total_sqft,0.575559,1.000000


### Creating the `price_per_sqft` Feature

A new column, `price_per_sqft`, is added to represent the price per square foot for each property. This feature enables more effective comparison of property values across different locations and sizes.

In [30]:
data['price_per_sqft'] = data['price']*100000 / data['total_sqft']

In [31]:
data['price_per_sqft']

0         3699.810606
1         4615.384615
2         4305.555556
3         6245.890861
4         4250.000000
             ...     
13315     6689.834926
13316    11111.111111
13317     5258.545136
13318    10407.336319
13319     3090.909091
Name: price_per_sqft, Length: 13320, dtype: float64

In [32]:
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,13274.000000,13320.000000,13320.000000,13320.000000,1.327400e+04
mean,1559.626694,2.688814,112.565627,2.802778,7.907501e+03
std,1238.405258,1.338754,148.971674,1.294496,1.064296e+05
min,1.000000,1.000000,8.000000,1.000000,2.678298e+02
25%,1100.000000,2.000000,50.000000,2.000000,4.266865e+03
50%,1276.000000,2.000000,72.000000,3.000000,5.434306e+03
75%,1680.000000,3.000000,120.000000,3.000000,7.311746e+03
max,52272.000000,40.000000,3600.000000,43.000000,1.200000e+07


In [33]:
data['location'].value_counts()

location
Whitefield                         541
Sarjapur  Road                     399
Electronic City                    302
Kanakpura Road                     273
Thanisandra                        234
                                  ... 
3rd Stage Raja Rajeshwari Nagar      1
Chuchangatta Colony                  1
Electronic City Phase 1,             1
Chikbasavanapura                     1
Abshot Layout                        1
Name: count, Length: 1305, dtype: int64

`location` columns preprocessing we will remove outlier if any.

In [34]:
# Remove leading and trailing spaces from location names
data['location'] = data['location'].apply(lambda x : x.strip())
location_count = data['location'].value_counts()


In [35]:
location_count  

location
Whitefield                              542
Sarjapur  Road                          399
Electronic City                         304
Kanakpura Road                          273
Thanisandra                             237
                                       ... 
Duddanahalli                              1
Doddanakunte                              1
Jogupalya                                 1
Subhash Nagar                             1
Kengeri Satellite Town KHB Apartment      1
Name: count, Length: 1294, dtype: int64

In [36]:
location_count_less_10 = location_count[location_count <= 10]
location_count_less_10

location
Basapura                                10
Dairy Circle                            10
Nagappa Reddy Layout                    10
Naganathapura                           10
Sector 1 HSR Layout                     10
                                        ..
Duddanahalli                             1
Doddanakunte                             1
Jogupalya                                1
Subhash Nagar                            1
Kengeri Satellite Town KHB Apartment     1
Name: count, Length: 1053, dtype: int64

Locations with 10 or fewer occurrences (totaling 1,053 unique locations) are replaced with `'other'`. This reduces the number of unique categories in the `location` column, simplifying encoding and improving model performance.

In [37]:

data['location'] = data['location'].apply(lambda x: 'other' if x in location_count_less_10 else x)

In [38]:
data['location'].value_counts()

location
other                        2885
Whitefield                    542
Sarjapur  Road                399
Electronic City               304
Kanakpura Road                273
                             ... 
Tindlu                         11
Marsur                         11
2nd Phase Judicial Layout      11
Thyagaraja Nagar               11
HAL 2nd Stage                  11
Name: count, Length: 242, dtype: int64

In [39]:
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,13274.000000,13320.000000,13320.000000,13320.000000,1.327400e+04
mean,1559.626694,2.688814,112.565627,2.802778,7.907501e+03
std,1238.405258,1.338754,148.971674,1.294496,1.064296e+05
min,1.000000,1.000000,8.000000,1.000000,2.678298e+02
25%,1100.000000,2.000000,50.000000,2.000000,4.266865e+03
50%,1276.000000,2.000000,72.000000,3.000000,5.434306e+03
75%,1680.000000,3.000000,120.000000,3.000000,7.311746e+03
max,52272.000000,40.000000,3600.000000,43.000000,1.200000e+07


Outlier Detection in `total_sqft`

A review of the `total_sqft` column reveals some unrealistic values, such as properties with as little as 1 square foot of area. Such entries are clear outliers or data entry errors, as it is not feasible for any property—especially those with multiple bedrooms—to have such a small area.

To improve data quality and ensure reliable analysis, we remove properties where the average area per BHK (i.e., `total_sqft` divided by `bhk`) is less than 300 sqft. This threshold helps filter out likely outliers and data inconsistencies, resulting in a more accurate and trustworthy dataset for further exploration and modeling.

In [40]:
data = data[((data['total_sqft']/data['bhk']) >= 300)]
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,12530.000000,12530.000000,12530.000000,12530.000000,12530.000000
mean,1594.564544,2.559537,111.382401,2.650838,6303.979357
std,1261.271296,1.077938,152.077329,0.976678,4162.237981
min,300.000000,1.000000,8.440000,1.000000,267.829813
25%,1116.000000,2.000000,49.000000,2.000000,4210.526316
50%,1300.000000,2.000000,70.000000,3.000000,5294.117647
75%,1700.000000,3.000000,115.000000,3.000000,6916.666667
max,52272.000000,16.000000,3600.000000,16.000000,176470.588235


In [41]:
data.shape

(12530, 7)

In [42]:
def remove_outliers_sqft(df):
    df_output = pd.DataFrame()
    for key,subdf in df.groupby('location'):
        
        m = np.mean(subdf['price_per_sqft'])

        st = np.std(subdf['price_per_sqft'])

        gen_df = subdf[(subdf['price_per_sqft'] > (m-st)) & (subdf['price_per_sqft'] <= (m+st))]
        
        df_output = pd.concat([df_output,gen_df],ignore_index= True)
       
    return df_output
data = remove_outliers_sqft(data)
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,10301.000000,10301.000000,10301.000000,10301.000000,10301.000000
mean,1508.440608,2.471702,91.286372,2.574896,5659.062876
std,880.694214,0.979449,86.342786,0.897649,2265.774749
min,300.000000,1.000000,10.000000,1.000000,1250.000000
25%,1110.000000,2.000000,49.000000,2.000000,4244.897959
50%,1286.000000,2.000000,67.000000,2.000000,5175.600739
75%,1650.000000,3.000000,100.000000,3.000000,6428.571429
max,30400.000000,16.000000,2200.000000,16.000000,24509.803922


In [43]:
data.shape

(10301, 7)

### Understanding `groupby` and Outlier Removal Logic

- **`groupby('location')`** splits the DataFrame into groups based on unique values in the `location` column.
    - **`key`**: Stores the name of the current location (a string, e.g., `'Whitefield'`).
    - **`subdf`**: Contains a DataFrame with all rows for that location.

- For each location, the function keeps only those rows where `price_per_sqft` is within one standard deviation of the mean for that location. This filters out unusually high or low property prices, keeping only the most typical values for each area.


In [44]:


def bhk_outlier_remove(df):
    exclude_indices = np.array([])

    for location, location_df in df.groupby('location'):
        bhk_stats = {}

        # Step 1: Calculate mean and std for each BHK in a location
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df['price_per_sqft']),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
  
        # Step 2: Compare each BHK price_per_sqft with the (BHK-1) stats
        
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk - 1)
            if stats and stats['count'] > 5:
                exclude_indices = np.append(
                    exclude_indices,
                    bhk_df[bhk_df['price_per_sqft'] < stats['mean']].index.values
                )

    return df.drop(exclude_indices, axis='index')


### Understanding and Removing BHK Outliers

The `bhk_outlier_remove` function refines real estate data by removing properties where the `price_per_sqft` is unusually low compared to properties with fewer rooms (BHK) in the same location. For example, it addresses cases where a 3BHK is listed at a lower price per square foot than a 2BHK nearby, which is typically unrealistic.

#### How It Works: Two-Step Process Per Location

**1. Calculate BHK Statistics**  
For each unique location, properties are grouped by their BHK count. For every BHK group within a location, the function computes:
- Mean `price_per_sqft`
- Standard deviation of `price_per_sqft`
- Count of properties for that BHK type

This builds a statistical profile for each BHK size in every location.

**2. Compare and Exclude Outliers**  
For each BHK type in a location:
- The function looks up the mean `price_per_sqft` of the (BHK - 1) group (e.g., for 3BHK, it checks 2BHK stats).
- If a property's `price_per_sqft` is less than the mean of the smaller BHK group (and the smaller group has more than 5 properties), it is flagged as an outlier.
- The indices of these outlier properties are collected.

**Final Step: Clean the DataFrame**  
After processing all locations and BHK groups, the function removes all identified outlier rows, resulting in a cleaner dataset for further analysis and modeling.


In [45]:
data = bhk_outlier_remove(data)

In [46]:
data.shape

(7361, 7)

In [47]:
data

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,1st Block Jayanagar,4 BHK,2850.0,4.0,428.0,4,15017.543860
1,1st Block Jayanagar,3 BHK,1630.0,3.0,194.0,3,11901.840491
2,1st Block Jayanagar,3 BHK,1875.0,2.0,235.0,3,12533.333333
3,1st Block Jayanagar,3 BHK,1200.0,2.0,130.0,3,10833.333333
4,1st Block Jayanagar,2 BHK,1235.0,2.0,148.0,2,11983.805668
...,...,...,...,...,...,...,...
10292,other,2 BHK,1200.0,2.0,70.0,2,5833.333333
10293,other,1 BHK,1800.0,1.0,200.0,1,11111.111111
10296,other,2 BHK,1353.0,2.0,110.0,2,8130.081301
10297,other,1 Bedroom,812.0,1.0,26.0,1,3201.970443


In [48]:
data.drop(columns=['size','price_per_sqft'],inplace=True)

### Cleaned data

In [49]:
data.head()

,location,total_sqft,bath,price,bhk
0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,1st Block Jayanagar,1875.0,2.0,235.0,3
3,1st Block Jayanagar,1200.0,2.0,130.0,3
4,1st Block Jayanagar,1235.0,2.0,148.0,2


In [50]:
data.to_csv("Cleaned_data.csv")

In [51]:
X = data.drop(columns=['price'])
y = data['price']


## Model Building 

The necessary libraries for model building and evaluation have been imported, including tools for data preprocessing, regression algorithms (Linear Regression, Lasso, Ridge), and performance metrics. This sets the stage for splitting the data, encoding categorical features, scaling, and training regression models.
```

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression , Lasso,Ridge
from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score


### Train-Test Split

The dataset is split into training and testing sets using an 80-20 ratio. This ensures that model evaluation is performed on unseen data, providing a reliable measure of performance.


In [53]:
X_train,X_test,y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state =0)

In [54]:
print(X_train.shape)
print(X_test.shape)

(5888, 4)
(1473, 4)


### Applying Linear Regression

In [55]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

# Create column transformer
column_trans = make_column_transformer(
    (OneHotEncoder(sparse_output=False), ['location']),
    remainder='passthrough'
)


The `column_trans` object is a column transformer that applies one-hot encoding to the `location` column, converting categorical location values into binary features. All other columns are left unchanged (`remainder='passthrough'`). This prepares the data for machine learning models that require numerical input.


In [56]:
scaler = StandardScaler()
lr = LinearRegression() 

In [57]:
pipe = make_pipeline(column_trans, scaler, lr)

The `pipe` object is a machine learning pipeline that sequentially applies the following steps:

1. **`column_trans`**: Transforms the `location` column using one-hot encoding and passes through other features unchanged.
2. **`scaler`**: Standardizes numerical features to have zero mean and unit variance.
3. **`lr`**: Fits a Linear Regression model to the processed data.

This pipeline streamlines preprocessing and model training into a single, reusable workflow.

In [58]:
pipe.fit(X_train, y_train)

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['location'])])),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

Fits the pipeline to the training data, performing preprocessing and training the Linear Regression model.

In [59]:
y_pred_lr = pipe.predict(X_test)

Performs predictions on the test set using the trained Linear Regression pipeline, generating estimated property prices for evaluation.

In [60]:
r2 = r2_score(y_test, y_pred_lr)
print("R² Score:", r2)

R² Score: 0.825197701638146


### Applying Lasso

In [61]:
lasso = Lasso()

In [62]:

pipe = make_pipeline(column_trans, scaler,lasso)

Creates a pipeline with column transformer, scaler, and Lasso regression for modeling.

In [63]:
pipe.fit(X_train,y_train)

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['location'])])),
                ('standardscaler', StandardScaler()), ('lasso', Lasso())])

Fits the pipeline to the training data, performing preprocessing and training the Linear Regression model.


Predicts property prices using the Lasso regression pipeline and evaluates model performance with the R² score.


In [64]:
y_pred_lasso = pipe.predict(X_test)
r2_score(y_test,y_pred_lasso)

0.814689475169039

### Applying Ridge

In [65]:
ridge = Ridge()

In [66]:
pipe = make_pipeline(column_trans,scaler,ridge)

Creates a pipeline with column transformer, scaler, and Ridge regression for modeling.

In [67]:
pipe.fit(X_train,y_train)

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['location'])])),
                ('standardscaler', StandardScaler()), ('ridge', Ridge())])

Fits the pipeline to the training data, performing preprocessing and training the Ridge Regression model.

Evaluating the Ridge regression model on the test set using the R² score.

In [68]:
y_pred_ridge = pipe.predict(X_test)
r2_score(y_test,y_pred_ridge)

0.825234850229012

In [69]:
print("No Regularisation :" ,r2_score(y_test,y_pred_lr))
print("Lasso :" ,r2_score(y_test,y_pred_lasso))
print("Ridge :" ,r2_score(y_test,y_pred_ridge))


No Regularisation : 0.825197701638146
Lasso : 0.814689475169039
Ridge : 0.825234850229012


## Model Comparison and Final Selection

The R² scores for the different regression models are as follows:

- **Linear Regression (No Regularisation):** 0.827
- **Lasso Regression:** 0.813
- **Ridge Regression:** 0.828

Both Linear and Ridge regression models perform similarly and outperform Lasso regression. Given Ridge regression's ability to handle multicollinearity and prevent overfitting through regularization, we will proceed with **Ridge Regression** as our final model for property price prediction.

In [70]:
# train_model_and_save.py

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
import joblib

# Load and clean data
df = pd.read_csv("Cleaned_data.csv")
if "Unnamed: 0" in df.columns:
    df.drop(columns=["Unnamed: 0"], inplace=True)

X = df.drop(columns=["price"])
y = df["price"]

# Create preprocessing + model pipeline
column_trans = make_column_transformer(
    (OneHotEncoder(sparse_output=False, handle_unknown="ignore"), ["location"]),
    remainder="passthrough"
)
pipeline = make_pipeline(column_trans, StandardScaler(), Ridge())

# Train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

# Save model and location list
joblib.dump(pipeline, "bengaluru_house_model.pkl")
joblib.dump(X["location"].unique().tolist(), "location_list.pkl")
print("✅ Model and location list saved successfully.")


✅ Model and location list saved successfully.


## Streamlit app

In [71]:
#  c:\Users\hp\Documents\ds\ds_materials\8.1_streamlit\streamlit_tutorial_in_1_class.ipynb

## using save model

In [72]:
# bengaluru_app.py

import streamlit as st
import pandas as pd
import numpy as np
import joblib

# Load model and location list
model = joblib.load("bengaluru_house_model.pkl")
locations = joblib.load("location_list.pkl")

# Streamlit UI
st.set_page_config(page_title="Bengaluru House Price Predictor", layout="centered")
st.title("🏠 Bengaluru House Price Predictor")

st.sidebar.header("Input House Details")

location = st.sidebar.selectbox("Select Location", sorted(locations))
total_sqft = st.sidebar.number_input("Total Square Feet", min_value=300.0, max_value=10000.0, value=1000.0)
bhk = st.sidebar.slider("Bedrooms (BHK)", 1, 10, 2)
bath = st.sidebar.slider("Bathrooms", 1, 10, 2)

if st.sidebar.button("Predict Price"):
    input_data = pd.DataFrame({
        "location": [location],
        "total_sqft": [total_sqft],
        "bhk": [bhk],
        "bath": [bath]
    })
    
    prediction = model.predict(input_data)[0]
    st.success(f"💰 Estimated Price: ₹{prediction:.2f} Lakhs")

st.markdown("---")
st.caption("Built with ❤️ using Streamlit and Scikit-Learn")


ModuleNotFoundError: No module named 'streamlit'

## 🟢 How to Run This App

1. Run model training:

   ```bash
   python train_model_and_save.py
   ```

2. Start Streamlit app:

   ```bash
   streamlit run bengaluru_app.py
   ```

---

## 📦 Optional: `requirements.txt`

```txt
streamlit
pandas
scikit-learn
joblib
```

---

## Without model save 

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
import joblib

# Load cleaned data
data = pd.read_csv("Cleaned_data.csv")
data.drop(columns=["Unnamed: 0"], inplace=True)

# Prepare features and target
X = data.drop(columns=["price"])
y = data["price"]

# Create column transformer
column_trans = make_column_transformer(
    (OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ['location']),
    remainder='passthrough'
)

# Create Ridge pipeline
scaler = StandardScaler()
ridge = Ridge()
pipe = make_pipeline(column_trans, scaler, ridge)

# Train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
pipe.fit(X_train, y_train)

# Streamlit App
st.title("Bengaluru House Price Predictor")

# Sidebar inputs
st.sidebar.header("User Input Features")
location = st.sidebar.selectbox("Location", sorted(data["location"].unique()))
total_sqft = st.sidebar.number_input("Total Sqft Area", min_value=300.0, max_value=10000.0, value=1000.0)
bhk = st.sidebar.slider("Number of Bedrooms (BHK)", 1, 10, 2)
bath = st.sidebar.slider("Number of Bathrooms", 1, 10, 2)

# Predict button
if st.sidebar.button("Predict Price"):
    input_df = pd.DataFrame({
        'location': [location],
        'total_sqft': [total_sqft],
        'bath': [bath],
        'bhk': [bhk]
    })
    prediction = pipe.predict(input_df)[0]
    st.success(f"Predicted House Price: ₹{round(prediction, 2)} Lakhs")

# Dataset preview
if st.checkbox("Show Cleaned Dataset"):
    st.dataframe(data.head(20))

# Model performance
st.markdown("---")
st.subheader("Model Evaluation")
r2_score = pipe.score(X_test, y_test)
st.write(f"R² Score on Test Set: {r2_score:.4f}")
